In [68]:
import numpy as np
import matplotlib.pyplot as plt
import itertools as its
import sys
import time
import os
import string
import random


import keras
import keras.backend as K
import keras.preprocessing.sequence as pps
import keras.layers.wrappers as wrappers

from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import RMSprop

In [54]:

# We get rid of most possible characters:
chars_to_keep = string.ascii_lowercase  + string.digits + ' ' + ',' + '.' + '?' + '!' + '(' + ')' + '/' + ':' + '-' + '\"' + '\''

char_to_idx = { c : i for i, c in enumerate(chars_to_keep)}
idx_to_char = {i : c for i, c in enumerate(chars_to_keep)}



def string_to_array(s):
    # map maybe_char() to input to remove uncomon characters:
    def maybe_char(c):
        if c in chars_to_keep:
            return c
        return ""
    # Return a numpy array:
    return np.array([char_to_idx[c] for c in "".join(map(maybe_char, s.lower()))], np.int8)



def get_data(folder='aclImdb', files_to_use=0.1, train_data=True):
    # Takes the first files_to_use files from both folders (max 25000 of each) and pads with zeros to max_len
    
    if train_data:
        folder += '/train'
    else:
        folder += '/test'
        
    X = []
    sub_folders = ['pos', 'neg']
    for sub_folder in sub_folders:
        path = folder + '/' + sub_folder + "/"
        files = os.listdir(path)
        
        # Only take the fraction files_to_use 
        files = files[:int(len(files)*files_to_use)]

        for fn in files:

            try:
                f = file(path + fn, 'r')
                ss = f.read()
                f.close()
            except Exception as e:
                print "Could not open file %s, error: %s %s %s"% (fn, type(e), e.__str__(), sys.exc_info()[0])
            
            X.append(string_to_array(ss))

    corpus = np.concatenate(X)
    
    return corpus


    

In [55]:
corpus = get_data()

In [56]:
maxlen = 10
chunks = []
next_chars = []

for i in range(0, corpus.shape[0] - maxlen - 1):
    chunks.append(corpus[i: i + maxlen])
    next_chars.append(corpus[i + maxlen])



In [57]:

X = np.zeros((len(chunks), maxlen, len(chars_to_keep)), dtype=np.bool)
y = np.zeros((len(chunks), len(chars_to_keep)), dtype=np.bool)
for i, chunk in enumerate(chunks):
    for t, char in enumerate(chunk):
        X[i, t, char] = 1
    y[i, next_chars[i]] = 1



In [58]:

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars_to_keep))))
model.add(Dense(len(chars_to_keep), activation='softmax'))



In [69]:

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


In [71]:

# train the model, output generated text after each iteration
model.fit(X, y,
          batch_size=128,
          epochs=1)


Epoch 1/1
3152861/3152861 [==============================] - 1547s - loss: 1.5930  


In [81]:

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


start_index = random.randint(0, len(corpus) - maxlen - 1)

temperature = 1.0

chunk = corpus[start_index: start_index + maxlen]
text = "".join(map(lambda k: idx_to_char[k], chunk))

x = np.zeros((1, maxlen, len(chars_to_keep)))

for t, char in enumerate(text):
    x[0, t, char_to_idx[char]] = 1



sys.stdout.write(text)

for i in range(400):
    x = np.zeros((1, maxlen, len(chars_to_keep)))
    for t, char in enumerate(text):
        x[0, t, char_to_idx[char]] = 1

    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, temperature)
    next_char = idx_to_char[next_index]

    text = text[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()

print("\n")
print("\n")




my favourite kashets and the custed by plyel millicane of the point. i have been shet have as chappen that it's leaving of marties thirk i very innicences out sometice, and def so get to make she were never more of sows to the deteated by and might remember as a film of trum lost of rest were amazing to how we read this cower watch, and sucklarachnion, a cast over and away a real fre solries kids for lausye





In [42]:
X = X[:3655053]
y = y[:3655053]

(10965161, 10, 48)

(3655053, 48)

32895484